ข้อมูลที่ publish เข้าสู่ Kafka broker คือ 2022-01-01-2022-12-31.csv ซึ่งมีหน้าตาแบบนี้

{"locationId": "225577", "location": "Bank of Ayuthaya Head Office Yan Nawa, Bangkok", "parameter": "pm25", "value": "35.0", "date": "{'utc': '2022-12-31T04:00:00+00:00', 'local': '2022-12-31T11:00:00+07:00'}", "unit": "\u00b5g/m\u00b3", "coordinates": "{'latitude': 13.679226, 'longitude': 100.54687}", "country": "NA", "city": "", "isMobile": "False", "isAnalysis": "", "entity": "Governmental Organization", "sensorType": "reference grade"}
{"locationId": "225577", "location": "Bank of Ayuthaya Head Office Yan Nawa, Bangkok", "parameter": "pm25", "value": "36.0", "date": "{'utc': '2022-12-31T05:00:00+00:00', 'local': '2022-12-31T12:00:00+07:00'}", "unit": "\u00b5g/m\u00b3", "coordinates": "{'latitude': 13.679226, 'longitude': 100.54687}", "country": "NA", "city": "", "isMobile": "False", "isAnalysis": "", "entity": "Governmental Organization", "sensorType": "reference grade"}
{"locationId": "225577", "location": "Bank of Ayuthaya Head Office Yan Nawa, Bangkok", "parameter": "pm25", "value": "36.0", "date": "{'utc': '2022-12-31T06:00:00+00:00', 'local': '2022-12-31T13:00:00+07:00'}", "unit": "\u00b5g/m\u00b3", "coordinates": "{'latitude': 13.679226, 'longitude': 100.54687}", "country": "NA", "city": "", "isMobile": "False", "isAnalysis": "", "entity": "Governmental Organization", "sensorType": "reference grade"}

In [1]:
# จำเป็นต้องเปลี่ยนเลขไอพีของ Kafka broker โดยใช้ INTERNAL IP Address พร้อมด้วยหมายเลข port 9092

kafka_broker = "10.128.0.12:9092"

In [2]:
# สร้าง Spark session โดยใช้การตั้งค่าที่กำหนด

from pyspark.sql import SparkSession

spark = SparkSession.\
        builder.\
        appName("KafkaSubscribe").\
        master("spark://spark-master:7077").\
        config("spark.executor.memory", "1000m").\
        config("spark.executor.cores", "2").\
        config("spark.cores.max", "6").\
        config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0").\
        getOrCreate()



Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/lib/python3.9/dist-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-a409be4f-74b2-4e10-9aa7-e90212da050c;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
:: resolution report :: resolve 668ms :: artifacts dl 12

# 1. ได้รับ Unbounded input table 

In [3]:
# สร้าง DataFrame โดยอ่านข้อมูลจาก Kafka stream

unboundInput_table_df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", kafka_broker) \
  .option("subscribe", "quickstart-events") \
  .option("group.id", "Aekanun-Spark-App") \
  .load()

In [4]:
# แสดง schema ของ column ต่างๆ ใน DataFrame

unboundInput_table_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



# 2. เกิดการ Query - Stream Data Processing

# 3. ได้รับ Result table

In [5]:
result_table_df = unboundInput_table_df

In [6]:
result_table_df.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



# 4. ได้รับ Output (table)

In [9]:
from pyspark.sql import functions as F



output_df = result_table_df.writeStream \
.queryName("allday")\
  .outputMode("append")\
  .format("memory")\
  .start()

output_df.awaitTermination(60)

23/06/22 13:27:51 WARN StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-42310af6-0411-4f4d-906f-5d431de059c9. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.


False

In [14]:
spark.sql('select * from allday').write.mode('overwrite').parquet('./outputParquet')

In [15]:
! ls -l /

total 76
drwxr-xr-x   1 root root 4096 Aug  5  2022 bin
drwxr-xr-x   2 root root 4096 Jun 30  2022 boot
drwxr-xr-x   5 root root  340 Jun 22 13:02 dev
drwxr-xr-x   1 root root 4096 Jun 21 11:10 etc
drwxr-xr-x   2 root root 4096 Jun 30  2022 home
drwxr-xr-x   1 root root 4096 Aug  5  2022 lib
drwxr-xr-x   2 root root 4096 Aug  1  2022 lib64
drwxr-xr-x   2 root root 4096 Aug  1  2022 media
drwxr-xr-x   2 root root 4096 Aug  1  2022 mnt
drwxr-xr-x   1 root root 4096 Aug  5  2022 opt
drwxr-xr-x   2 root root 4096 Jun 22 13:31 outputParquet
dr-xr-xr-x 212 root root    0 Jun 22 13:02 proc
drwx------   1 root root 4096 Jun 21 12:51 root
drwxr-xr-x   3 root root 4096 Aug  1  2022 run
drwxr-xr-x   2 root root 4096 Aug  1  2022 sbin
drwxr-xr-x   2 root root 4096 Aug  1  2022 srv
dr-xr-xr-x  13 root root    0 Jun 22 13:02 sys
drwxrwxrwt   1 root root 4096 Jun 22 13:27 tmp
drwxr-xr-x   1 root root 4096 Aug  1  2022 usr
drwxr-xr-x   1 root root 4096 Aug  1  2022 var
